In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.utils.data as data

from datasets import load_dataset
from src.landscape import Landscape
from src.trainer import Trainer
from src.plot import Plot

In [ ]:
def pre_process(example):
    arr = np.reshape(example["input"], -1)
    example["input"] = arr
    return example

In [ ]:
mnist = load_dataset("mnist", trust_remote_code=True)
train, test = mnist.get("train"), mnist.get("test")

In [ ]:
train.set_format(type="numpy", columns=["image", "label"])
test.set_format(type="numpy", columns=["image", "label"])
train = train.rename_column("image", "input")
test = test.rename_column("image", "input")
train = train.map(pre_process, num_proc=4)
test = test.map(pre_process, num_proc=4)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
train_inputs = torch.from_numpy(train["input"]).float().squeeze() / 255.0
test_inputs = torch.from_numpy(test["input"]).float().squeeze() / 255.0
train_labels = torch.from_numpy(train["label"]).long()
test_labels = torch.from_numpy(test["label"]).long()

In [ ]:
train_dataset = data.TensorDataset(train_inputs, train_labels)
test_dataset = data.TensorDataset(test_inputs, test_labels)

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_1 = nn.Linear(28 * 28, 512)
        self.relu_1 = nn.ReLU()
        self.drop_1 = nn.Dropout(p=0.4)
        self.linear_2 = nn.Linear(512, 512)
        self.relu_2 = nn.ReLU()
        self.drop_2 = nn.Dropout(p=0.2)
        self.linear_3 = nn.Linear(512, 512)
        self.relu_3 = nn.ReLU()
        self.linear_4 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.drop_1(self.relu_1(self.linear_1(x)))
        x = self.drop_2(self.relu_2(self.linear_2(x)))
        x = self.relu_3(self.linear_3(x))
        out = self.linear_4(x)
        return out

In [ ]:
model = Model()
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()
batch_size = 256

In [ ]:
train_loader = data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=False,
    pin_memory=True,
)

test_loader = data.DataLoader(
    test_dataset,
    batch_size=len(test_dataset),
    shuffle=True,
    drop_last=False,
    pin_memory=True,
)

In [ ]:
trainer = Trainer(model, optimizer, loss_fn, write=True, filepath="./metrics.h5", modelpath="./params.pt", trajpath="./traj.pt")

In [ ]:
metrics = trainer.train(train_loader, test_loader, epochs=10, device=device, printevery=10)

In [ ]:
model = Model()
landscape = Landscape.fromfiles(model, modelpath="./params.pt", trajpath="./traj.pt")

In [ ]:
# A, B, Z = landscape.create(loss_fn, test_loader, resolution=15, bounds=(-1.0, 1.0), device=device, printevery=15)

In [ ]:
# plot = Plot.fromfiles(llspath="./landscape.h5")

In [ ]:
# plot.plotsurface3D(filepath="./loss-landscape.png")

In [ ]:
landscape.pca()